In [ ]:
# TODO 
# deploy as a t2
# try only returning a single item from the result tuple and return it as a list

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.vision import *
from fastai.callbacks import SaveModelCallback, EarlyStoppingCallback
import numpy as np
import pandas as pd

import torch
from  torch import nn

In [3]:
images_path = '../data/resized_images'
labels_path = '../data/resized_labels.csv'
target_size = 320

In [4]:
y = pd.read_csv(labels_path, index_col='filename')

In [5]:
def get_y_func(x):
    filename = str(x).split('/')[-1]
    coord_list = []
    coords = list(y.loc[filename])
    for i in range(len(coords)//2):
        coord_list.append([coords[i*2+1],coords[i*2]])
    return tensor(coord_list)

In [6]:
transforms = get_transforms(
    do_flip=False,
    max_rotate=45, 
    max_zoom=1.5, 
    max_lighting=0.5, 
    max_warp=0.4, 
    p_affine=1., 
    p_lighting=1.
) 

In [7]:
data = (
    PointsItemList
        .from_folder(images_path)
        .split_by_rand_pct(0.15)
        .label_from_func(get_y_func)
        .transform(
            transforms,
            size=320,
            tfm_y=True, 
            remove_out=False, 
            padding_mode='reflection', 
            resize_method=ResizeMethod.PAD
        )
        .databunch()
        .normalize(imagenet_stats)
)

In [8]:
# data.show_batch(3, figsize=(6,6))

In [9]:
learn = cnn_learner(
    data, 
    models.resnet34, 
    loss_func=MSELossFlat()
)#.to_fp16()

In [10]:
learn.path = Path('..')

In [11]:
# learn.lr_find()
# learn.recorder.plot()

In [12]:
# learn.freeze_to(-1)
# learn.fit_one_cycle(
#     30, 
#     slice(1e-2), 
#     callbacks=[
#         SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='resnet34_frozen_to_-1'),
#         EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.01, patience=3)
#     ]
# )

In [13]:
# learn = learn.load('fastai_model')
# learn = learn.to_fp32()
# learn.save('fastai_model')
# learn.export('models/export.pkl')

In [14]:
learn = load_learner('../models', 'export.pkl')

In [15]:
# learn.show_results(rows=10)

### Bentoml Route

In [16]:
%%writefile pet_regression.py
from bentoml import BentoService, api, env, artifacts 
from bentoml.artifact import FastaiModelArtifact
from bentoml.handlers import FastaiImageHandler

@env(pip_dependencies=['gevent', 'pillow==6.2.2', 'fastai'])
@artifacts([FastaiModelArtifact('pet_regressor')])
class PetRegression(BentoService):
    
    @api(FastaiImageHandler)
    def predict(self, image):
        result = self.artifacts.pet_regressor.predict(image)
        return result

Overwriting pet_regression.py


In [17]:
# 1) import the custom BentoService defined above
from pet_regression import PetRegression

# 2) `pack` it with required artifacts
service = PetRegression.pack(pet_regressor=learn)

# 3) save your BentoSerivce
saved_path = service.save()

[2020-01-14 00:45:06,243] INFO - BentoService bundle 'PetRegression:20200114004452_4E03AE' created at: /tmp/bentoml-temp-02gh0dvb
[2020-01-14 00:45:06,563] INFO - BentoService bundle 'PetRegression:20200114004452_4E03AE' created at: /home/ec2-user/bentoml/repository/PetRegression/20200114004452_4E03AE


In [18]:
from bentoml import load

service = load(saved_path)

print(service.predict(data.get(0)))

[2020-01-14 00:45:15,154] WARNING - Module `pet_regression` already loaded, using existing imported module.
(ImagePoints (320, 320), tensor([[-0.5133, -0.5090],
        [-0.7960, -0.4386],
        [-0.8005, -0.2425],
        [-0.8656, -0.0839],
        [-0.5567, -0.0082],
        [-0.2026, -0.2832],
        [-0.5261, -0.3297],
        [-0.5509, -0.1878],
        [-0.2745, -0.3370],
        [-0.3003, -0.2733],
        [-0.3027, -0.2223]]), tensor([-0.5133, -0.5090, -0.7960, -0.4386, -0.8005, -0.2425, -0.8656, -0.0839,
        -0.5567, -0.0082, -0.2026, -0.2832, -0.5261, -0.3297, -0.5509, -0.1878,
        -0.2745, -0.3370, -0.3003, -0.2733, -0.3027, -0.2223]))


In [19]:
!pip install --upgrade {saved_path}

Processing /home/ec2-user/bentoml/repository/PetRegression/20200114004452_4E03AE
  Created wheel for PetRegression: filename=PetRegression-20200114004452_4E03AE-cp36-none-any.whl size=51633239 sha256=260d38a585b0df2d3c65f6f2292f54ea65e578d1d3fbc7b1617fcdb59c28d14f
  Stored in directory: /tmp/pip-ephem-wheel-cache-0d5f25up/wheels/8a/14/98/61b585f9ea979672f580cfad99359cdaa48f9cb67e6d8d1462
Successfully built PetRegression
  Found existing installation: PetRegression 20200113231107.post817695
    Uninstalling PetRegression-20200113231107.post817695:
      Successfully uninstalled PetRegression-20200113231107.post817695


In [20]:
!PetRegression predict --input=../data/resized_images/Abyssinian_1.jpg

(ImagePoints (320, 320), tensor([[-0.4128,  0.1322],
        [-0.7365,  0.1464],
        [-0.6189,  0.2665],
        [-0.7248,  0.4663],
        [-0.4005,  0.3247],
        [-0.2695,  0.2497],
        [-0.3914,  0.2126],
        [-0.4023,  0.3228],
        [-0.2775,  0.2574],
        [-0.2962,  0.2582],
        [-0.2722,  0.2691]]), tensor([-0.4128,  0.1322, -0.7365,  0.1464, -0.6189,  0.2665, -0.7248,  0.4663,
        -0.4005,  0.3247, -0.2695,  0.2497, -0.3914,  0.2126, -0.4023,  0.3228,
        -0.2775,  0.2574, -0.2962,  0.2582, -0.2722,  0.2691]))


In [21]:
bento_tag = '{name}:{version}'.format(name=service.name, version=service.version)
print(bento_tag)

PetRegression:20200114004452_4E03AE


In [22]:
!bentoml deployment create facial-features-detector --bento {bento_tag} --platform aws-sagemaker --api-name predict --region us-east-1

[2020-01-14 00:46:05,343] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-01-14 00:46:05,343] INFO - 

[2020-01-14 00:46:05,344] INFO -  ---> 406f2b43ea59

[2020-01-14 00:46:05,344] INFO - Step 2/11 : EXPOSE 8080
[2020-01-14 00:46:05,344] INFO - 

[2020-01-14 00:46:05,344] INFO -  ---> Using cache

[2020-01-14 00:46:05,344] INFO -  ---> 5ab208181bd7

[2020-01-14 00:46:05,344] INFO - Step 3/11 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential     && apt-get install -y nginx      && rm -rf /var/lib/apt/lists/*
[2020-01-14 00:46:05,345] INFO - 

[2020-01-14 00:46:05,345] INFO -  ---> Using cache

[2020-01-14 00:46:05,345] INFO -  ---> d928e3a383d7

[2020-01-14 00:46:05,345] INFO - Step 4/11 : RUN conda install pip numpy scipy       && pip install gunicorn gevent
[2020-01-14 00:46:05,345] INFO - 

[2020-01-14 00:46:05,345] INFO -  ---> Using cache

[2020-01-14 00:46:05,345] INFO -  ---> 1eef532

In [45]:
!bentoml --verbose deployments delete facial-features-detector --force

[2020-01-14 01:39:27,757] DEBUG - Using BentoML with local Yatai server
[2020-01-14 01:39:28,059] DEBUG - Upgrading tables to the latest revision
[2020-01-14 01:39:29,504] DEBUG - AWS delete endpoint response: {'ResponseMetadata': {'RequestId': 'bc2727cd-d182-40a6-9337-30d496724061', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'bc2727cd-d182-40a6-9337-30d496724061', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Tue, 14 Jan 2020 01:39:29 GMT'}, 'RetryAttempts': 0}}
[2020-01-14 01:39:29,587] DEBUG - AWS delete model response: {'ResponseMetadata': {'RequestId': '554876ac-1c6f-4683-9552-6cf65e5171d5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '554876ac-1c6f-4683-9552-6cf65e5171d5', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Tue, 14 Jan 2020 01:39:28 GMT'}, 'RetryAttempts': 0}}
[2020-01-14 01:39:29,632] DEBUG - AWS delete endpoint config response: {'ResponseMetadata': {'RequestId': '64a02cf8-3

In [38]:
import json
import boto3
runtime = boto3.Session().client(service_name='sagemaker-runtime')

img = open('../data/resized_images/Abyssinian_1.jpg', 'rb').read()
    
response = runtime.invoke_endpoint(
    EndpointName='dev-facial-features-detector', 
    Body=bytearray(img),
)

result = response['Body'].read()
result = json.loads(result)

In [39]:
result['result']

'(ImagePoints (320, 320), tensor([[-0.4128,  0.1322],\n        [-0.7365,  0.1464],\n        [-0.6189,  0.2665],\n        [-0.7248,  0.4663],\n        [-0.4005,  0.3247],\n        [-0.2695,  0.2497],\n        [-0.3914,  0.2126],\n        [-0.4023,  0.3228],\n        [-0.2775,  0.2574],\n        [-0.2962,  0.2582],\n        [-0.2722,  0.2691]]), tensor([-0.4128,  0.1322, -0.7365,  0.1464, -0.6189,  0.2665, -0.7248,  0.4663,\n        -0.4005,  0.3247, -0.2695,  0.2497, -0.3914,  0.2126, -0.4023,  0.3228,\n        -0.2775,  0.2574, -0.2962,  0.2582, -0.2722,  0.2691]))'

In [40]:
str(learn.predict(data.get(0)))

'(ImagePoints (320, 320), tensor([[-0.5133, -0.5090],\n        [-0.7960, -0.4386],\n        [-0.8005, -0.2425],\n        [-0.8656, -0.0839],\n        [-0.5567, -0.0082],\n        [-0.2026, -0.2832],\n        [-0.5261, -0.3297],\n        [-0.5509, -0.1878],\n        [-0.2745, -0.3370],\n        [-0.3003, -0.2733],\n        [-0.3027, -0.2223]]), tensor([-0.5133, -0.5090, -0.7960, -0.4386, -0.8005, -0.2425, -0.8656, -0.0839,\n        -0.5567, -0.0082, -0.2026, -0.2832, -0.5261, -0.3297, -0.5509, -0.1878,\n        -0.2745, -0.3370, -0.3003, -0.2733, -0.3027, -0.2223]))'